In [55]:
import pandas as pd
import nltk 
from nltk.tokenize import word_tokenize  
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from string import punctuation
from math import log10
stop_words = stopwords.words('english')
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to /home/nik/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/nik/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [57]:
def train(df, size):
    lemmatizer = WordNetLemmatizer()
    X = df.copy()
    target = df['Category']
    X = X.drop('Category',axis = 1)
    train_X, test_X, train_y, test_y = train_test_split(X, target, test_size=size, stratify=target)
    train_X['Category'] = train_y
    test_X['Category'] = test_y
    tf = dict()
    for i in df["Category"].unique():
        tf[i] = {}
    cf = dict()
    result = {row.ArticleId: [row.Text, row.Category] for (index, row) in train_X.iterrows()}
    for i in result:
    #     print(i)
        result[i][0] = result[i][0].lower()
        result[i][0] = word_tokenize(result[i][0])
        no_stop_words = []                   
        for word in result[i][0]:                                   #iterating through every token.
            if word not in stop_words:                        
                no_stop_words.append(word)              #if the word is not a stop word append it.
        result[i][0] = no_stop_words
        no_punc = []
        for word in result[i][0]:                                   #iterating through every token.
            if word not in punctuation:
                for punctuationChar in punctuation :
                    word = word.replace(punctuationChar,'')
                no_punc.append(word)              #if the word is not a stop word append it.
        result[i][0] = no_punc

        word_list = result[i][0]
        res_list = []
        for word in word_list:
            res_list.append( lemmatizer.lemmatize(word))
        result[i][0] = res_list
    for i in result:
        word_list = result[i][0]
        target = result[i][1]
        for word in word_list:
            if word in tf[target].keys():
                tf[target][word] = tf[target][word] + 1
            else:
                tf[target][word] = 1
    maxi = 0
    for i in result:
        word_list = result[i][0]
        for word in word_list:
            count = 0
            for c in tf.keys():
                if word in tf[c].keys():
                    count = count + 1
            maxi = max(count,maxi)
            cf[word] = count

    tf_icf = tf.copy()
    for c in cf:
        for i in tf.keys():
            if c in tf[i].keys() and cf[c] != 0:
                tf_icf[i][c] = tf[i][c]*log10(5.0/cf[c])
    word_corpus = dict()
    for key in result.keys():
        for word in result[key][0]:
            word_corpus[word]=1
    conditionProb = {}
    for key in tf_icf.keys():
        conditionProb[key] = {}
    for c in cf:
        for key in tf_icf.keys():
            if key in tf_icf.keys() and c in tf_icf[key]:
                conditionProb[key][c] = (tf_icf[key][c] + 1)/(sum(tf_icf[key].values()) + len(word_corpus))
#                 conditionProb[key][c] = (tf_icf[key][c])/(sum(tf_icf[key].values()))
    
    total_sum = 0
    
    for i in tf.keys():
        count = 0
        prior_prob[i] = 0
        for c in result:
            target = result[c][1]
            if i == target:
                prior_prob[i] += 1
                total_sum += 1
    
    for i in tf.keys():
        prior_prob[i] = prior_prob[i]/total_sum
    mapping = dict()
    count = 0;
    for i in tf.keys():
        mapping[i] = count
        count = count + 1
    return test_X, conditionProb,prior_prob,mapping
    
    
    
def trainIDF(df, size):
    lemmatizer = WordNetLemmatizer()
    X = df.copy()
    target = df['Category']
    X = X.drop('Category',axis = 1)
    train_X, test_X, train_y, test_y = train_test_split(X, target, test_size=size, stratify=target)
    train_X['Category'] = train_y
    test_X['Category'] = test_y
    tf = dict()
    for i in df["Category"].unique():
        tf[i] = {}
    df = dict()
    result = {row.ArticleId: [row.Text, row.Category] for (index, row) in train_X.iterrows()}
    for i in result:
    #     print(i)
        result[i][0] = result[i][0].lower()
        result[i][0] = word_tokenize(result[i][0])
        no_stop_words = []                   
        for word in result[i][0]:                                   #iterating through every token.
            if word not in stop_words:                        
                no_stop_words.append(word)              #if the word is not a stop word append it.
        result[i][0] = no_stop_words
        no_punc = []
        for word in result[i][0]:                                   #iterating through every token.
            if word not in punctuation:
                for punctuationChar in punctuation :
                    word = word.replace(punctuationChar,'')
                no_punc.append(word)              #if the word is not a stop word append it.
        result[i][0] = no_punc

        word_list = result[i][0]
        res_list = []
        for word in word_list:
            res_list.append( lemmatizer.lemmatize(word))
        result[i][0] = res_list
    for i in result:
        word_list = result[i][0]
        target = result[i][1]
        for word in word_list:
            if word in tf[target].keys():
                tf[target][word] = tf[target][word] + 1
            else:
                tf[target][word] = 1
    maxi = 0
    df = dict()
    maxi = 0
    for i in result:
        word_list = result[i][0]
        for word in word_list:
            if word in df.keys():
                continue
            count = 0
            for c in result.keys():
                if word in result[c][0]:
                    count = count + 1
            maxi = max(count,maxi)
            df[word] = count
    
    word_corpus = dict()
    for key in result.keys():
        for word in result[key][0]:
            word_corpus[word]=1
    conditionProb = {}
    tf_idf = tf.copy()
    for key in tf_idf.keys():
        conditionProb[key] = {}
 
    for c in df:
        for i in tf.keys():
            if c in tf[i].keys() and df[c] != 0:
                tf_idf[i][c] = tf[i][c]*log10(len(X)/df[c])
    for c in df:
        for key in tf_idf.keys():
            if key in tf_idf.keys() and c in tf_idf[key]:
                conditionProb[key][c] = (tf_idf[key][c] + 1)/(sum(tf_idf[key].values()) + len(word_corpus))
    prior_prob = {}
    
    total_sum = 0
    
    for i in tf.keys():
        count = 0
        prior_prob[i] = 0
        for c in result:
            target = result[c][1]
            if i == target:
                prior_prob[i] += 1
                total_sum += 1
    
    for i in tf.keys():
        prior_prob[i] = prior_prob[i]/total_sum
    mapping = dict()
    count = 0;
    for i in tf.keys():
        mapping[i] = count
        count = count + 1
    return test_X, conditionProb,prior_prob,mapping
    
    

def predict(test_x,conditionProb,prior_prob,mapping):
    result1 = {row.ArticleId: [row.Text, row.Category] for (index, row) in test_X.iterrows()}
    for i in result1:
    #     print(i)
        result1[i][0] = result1[i][0].lower()
        result1[i][0] = word_tokenize(result1[i][0])
        no_stop_words = []                   
        for word in result1[i][0]:                                   #iterating through every token.
            if word not in stop_words:                        
                no_stop_words.append(word)              #if the word is not a stop word append it.
        result1[i][0] = no_stop_words
        no_punc = []
        for word in result1[i][0]:                                   #iterating through every token.
            if word not in punctuation:
                for punctuationChar in punctuation :
                    word = word.replace(punctuationChar,'')
                no_punc.append(word)              #if the word is not a stop word append it.
        result1[i][0] = no_punc
    for i in result1:
        word_list = result1[i][0]
        res_list = []
        for word in word_list:
            res_list.append( lemmatizer.lemmatize(word))
    result1[i][0] = res_list

    rows, cols = (5, 5)
    confusionMatrix = [ [ 0 for i in range(5) ] for j in range(5) ]
    

    count = 0
    for c in result1.keys():
        mini = 0
        pred = None
        for key in conditionProb.keys():
            y = 0
            for word in result1[c][0]:
                if word in conditionProb[key].keys():
                    y += log10(conditionProb[key][word])

                y += log10(prior_prob[key])
                if(y < mini):
                    mini = y
                    pred = key
        if pred == result1[c][1]:
            count = count + 1

        predicted = mapping[pred]
        actual = mapping[result1[c][1]]
        confusionMatrix[predicted][actual] += 1


    print("accuracy",count / len(result1.keys()))
    print("confusion matrix",confusionMatrix)
    num_classes = len(confusionMatrix)

    precision = []
    recall = []
    f1 = []
    for i in range(num_classes):
        tp = confusionMatrix[i][i]
        fp = sum([confusionMatrix[j][i] for j in range(num_classes) if j != i])
        fn = sum([confusionMatrix[i][j] for j in range(num_classes) if j != i])

        precision_i = tp / (tp + fp)
        recall_i = tp / (tp + fn)
        f1_i = 2 * (precision_i * recall_i) / (recall_i + precision_i)

        precision.append(precision_i)
        f1.append(f1_i)
        recall.append(recall_i)
    print("confusionMatrix",confusionMatrix)
    avg_precision = sum(precision) / num_classes
    avg_recall = sum(recall) / num_classes
    avg_f1 = sum(f1) / num_classes
    print("Precision: ", precision)
    print("Recall: ", recall)
    print("F1: ",f1)
    print("Average Precision: ", avg_precision)
    print("Average Recall: ", avg_recall)
    print("Average F1: ",avg_f1)
    


In [58]:
def main():
    df = pd.read_csv('BBC News Train.csv')
    print(df.head())
    print()
    print("TF-ICF 20% test")
    test_X, conditionProb,prior_prob,mapping = train(df,0.2)
    predict(test_X, conditionProb,prior_prob,mapping)
    print("TF-ICF 30% test")
    test_X, conditionProb,prior_prob,mapping = train(df,0.3)
    predict(test_X, conditionProb,prior_prob,mapping)
    print("TF-ICF 40% test")
    test_X, conditionProb,prior_prob,mapping = train(df,0.4)
    predict(test_X, conditionProb,prior_prob,mapping)
    print("TF-IDF 20% test")
    test_X, conditionProb,prior_prob,mapping = trainIDF(df,0.2)
    predict(test_X, conditionProb,prior_prob,mapping)
    print("TF-IDF 30% test")
    test_X, conditionProb,prior_prob,mapping = trainIDF(df,0.3)
    predict(test_X, conditionProb,prior_prob,mapping)
    print("TF-IDF 40% test")
    test_X, conditionProb,prior_prob,mapping = trainIDF(df,0.4)
    predict(test_X, conditionProb,prior_prob,mapping)
if __name__ == "__main__":
    main()


   ArticleId                                               Text  Category
0       1833  worldcom ex-boss launches defence lawyers defe...  business
1        154  german business confidence slides german busin...  business
2       1101  bbc poll indicates economic gloom citizens in ...  business
3       1976  lifestyle  governs mobile choice  faster  bett...      tech
4        917  enron bosses in $168m payout eighteen former e...  business

TF-ICF 20% test
accuracy 0.9748322147651006
confusion matrix [[125, 0, 0, 0, 0], [8, 103, 1, 0, 2], [2, 0, 109, 0, 0], [0, 0, 0, 137, 0], [0, 1, 0, 1, 107]]
confusionMatrix [[125, 0, 0, 0, 0], [8, 103, 1, 0, 2], [2, 0, 109, 0, 0], [0, 0, 0, 137, 0], [0, 1, 0, 1, 107]]
Precision:  [0.9259259259259259, 0.9903846153846154, 0.990909090909091, 0.9927536231884058, 0.981651376146789]
Recall:  [1.0, 0.9035087719298246, 0.9819819819819819, 1.0, 0.981651376146789]
F1:  [0.9615384615384615, 0.9449541284403671, 0.9864253393665158, 0.9963636363636363, 0.98165137